# Toward Certifiably Robust Face Recognition

- Seunghun Paik, Dongsoo Kim, Chanwoo Hwang, Sunpill Kim, and Jae Hong Seo
- Accepted for presentation at ECCV 2024

This is a notebook for reproducing the data reported in Table 1, 2

### Prerequisite

- You need to download the same test datasets we used: LFW, CFP-FP and AgedDB. The aligned version of them can be found by downloading one of the datasets uploaded in https://github.com/deepinsight/insightface/tree/master/recognition/_datasets_

### Notes for Reproducing Section E.
For reproducing the comparison with adversarailly trained models from RobFR, you need to downlaod the pre-trained weights from the following links

- PGD-Arc:
http://ml.cs.tsinghua.edu.cn/~dingcheng/ckpts/face_models/PGD_IR50_ArcFace/Backbone_IR_50_Epoch_16_Batch_30144_Time_2020-08-20-06-41_checkpoint.pth
- PGD-Cos:
http://ml.cs.tsinghua.edu.cn/~dingcheng/ckpts/face_models/PGD_IR50_CosFace/Backbone_IR_50_Epoch_32_Batch_60288_Time_2020-08-04-03-37_checkpoint.pth
- TRADES-Arc:
http://ml.cs.tsinghua.edu.cn/~dingcheng/ckpts/face_models/TRADES_IR50_ArcFace/Backbone_IR_50_Epoch_16_Batch_30144_Time_2020-09-03-09-58_checkpoint.pth
- TRADES-Cos:
http://ml.cs.tsinghua.edu.cn/~dingcheng/ckpts/face_models/TRADES_IR50_CosFace/Backbone_IR_50_Epoch_32_Batch_60288_Time_2020-09-06-04-48_checkpoint.pth

Since these backbones gets images with the pixel range of [0, 255] as inputs, appropriate rescaling is required. Since we already did this when defining the model, so you don't need to care about it.

In addition, their backbones are incompatible with those implemented in `insightface`. For this reason, we provide sourcdcodes in `/backbones/iresnet_AT.py` that were forked from the RobFR library.

### Step 1. Load Dataset and Backbone

In [ ]:
# Dataset
from eval.verification import load_bin
img_dir = "/* Your Directory for Test Datasets */"
lfw = load_bin(img_dir + "lfw.bin", (112,112))
cfp = load_bin(img_dir + "cfp_fp.bin", (112,112))
age = load_bin(img_dir + "agedb_30.bin", (112,112))
datasets = [("LFW", lfw), ("CFP-FP", cfp), ("AgeDB", age)]

In [ ]:
# Helper class for Re-scaling
import torch
from torch import nn
import torch.nn.functional as F

class Multiply2(nn.Module):
    def __init__(self):
        super().__init__()
        pass
    
    def forward(self, x):
        return 2 * x

device = "cuda:0"   # Your Device

# Backbone
from backbones.sllnet import SLLNet

# Note. Since our `iresnet50/100' was forked from Insightface, pre-trained parameters by theirs are compatible!
# See: https://github.com/deepinsight/insightface/tree/master/model_zoo
from backbones.iresnet import iresnet100

arcface = iresnet100()
arcface.load_state_dict(torch.load("/* Your Backbone */", map_location = "cpu"))
arcface = nn.Sequential(Multiply2(), arcface)  # Pixel Range in LFW is set to [-0.5, 0.5]
arcface.eval().to(device)

# Please download the pre-trained parameter from the link in README
proposed = SLLNet([3,4,6,3])
proposed.load_state_dict(torch.load("/* Your Backbone */", map_location ="cpu")) # Enclosed Parameter
proposed.eval().to(device)

# Models for AT
from backbones.iresnet_AT import IR_50
model_at = IR_50()
model_at.load_state_dict(torch.load("/* Your Backbone */", map_location = "cpu"))
model_at.eval().to(device)

print("Done!")

In [ ]:
# Quick Test for benchmarks
# Measuring TAR@FAR=1e-3
from eval_tools import feat_ext, benchmark_standard

# ArcFace
feat, norm = feat_ext(arcface, lfw, 128, device)
print("norm: ", norm)
tar, far, acc, thx = benchmark_standard(feat, 1e-3)
print("tar, far, acc, thx: ", tar, far, acc, thx)

# Proposed
feat, norm = feat_ext(proposed, lfw, 128, device)
print("norm: ", norm)
tar, far, acc, thx = benchmark_standard(feat, 1e-3)
print("tar, far, acc, thx: ", tar, far, acc, thx)


### Step 2. Table 1: Certifiable Robustness

In [ ]:
from eval_tools import benchmark_certified

# eps = 1.5
crau, crad, crai, epsu, epsd, epsi = benchmark_certified(lfw, proposed, 128, device, thx, 1.5)
print(f"CRA_U: {crau:.4f}\teps_U: {epsu:.3f}\tCRA_D: {crad:.4f}\teps_D: {epsd:.3f}\tCRA_I: {crai:.4f}\teps_I: {epsi:.3f}")

# eps = 3.0
crau, crad, crai, epsu, epsd, epsi = benchmark_certified(lfw, proposed, 128, device, thx, 3.0)
print(f"CRA_U: {crau:.4f}\teps_U: {epsu:.3f}\tCRA_D: {crad:.4f}\teps_D: {epsd:.3f}\tCRA_I: {crai:.4f}\teps_I: {epsi:.3f}")

# eps = 0.25
crau, crad, crai, epsu, epsd, epsi = benchmark_certified(lfw, proposed, 128, device, thx, 0.25)
print(f"CRA_U: {crau:.4f}\teps_U: {epsu:.3f}\tCRA_D: {crad:.4f}\teps_D: {epsd:.3f}\tCRA_I: {crai:.4f}\teps_I: {epsi:.3f}")

# eps = 0.75
crau, crad, crai, epsu, epsd, epsi = benchmark_certified(lfw, proposed, 128, device, thx, 0.75)
print(f"CRA_U: {crau:.4f}\teps_U: {epsu:.3f}\tCRA_D: {crad:.4f}\teps_D: {epsd:.3f}\tCRA_I: {crai:.4f}\teps_I: {epsi:.3f}")

### Step 3. Table 2: Empirical Robustness

#### Step 3-1. Attack Parameter Presets

In [ ]:
from easydict import EasyDict as edict

# Presets for PGD
paramPreset_PGD1 = edict()
paramPreset_PGD1.eps = 1.5
paramPreset_PGD1.alpha = 1.25
paramPreset_PGD1.n_iter = 20

paramPreset_PGD2 = edict()
paramPreset_PGD2.eps = 3.0
paramPreset_PGD2.alpha = 1.25
paramPreset_PGD2.n_iter = 20

paramPreset_PGD3 = edict()
paramPreset_PGD3.eps = 0.25
paramPreset_PGD3.alpha = 1.25
paramPreset_PGD3.n_iter = 20

paramPreset_PGD4 = edict()
paramPreset_PGD4.eps = 0.75
paramPreset_PGD4.alpha = 1.25
paramPreset_PGD4.n_iter = 20

# Presets for C&W
paramPreset_CW1 = edict()
paramPreset_CW1.alpha = 0.01
paramPreset_CW1.n_iter = 100
paramPreset_CW1.kappa1 = 0.05
paramPreset_CW1.kappa2 = 0
paramPreset_CW1.c = 3

paramPreset_CW2 = edict()
paramPreset_CW2.alpha = 0.01
paramPreset_CW2.n_iter = 100
paramPreset_CW2.kappa1 = 0.05
paramPreset_CW2.kappa2 = 0
paramPreset_CW2.c = 30

# Target Models
models = [
    ("ArcFace", arcface),
    ("Proposed", proposed),
]


#### Step 3-2. Test PGD Attack

In [ ]:
from attacks import PGD
from eval_tools import test_pgd

attack_configs = [
    ("U", PGD, paramPreset_PGD1),
    ("U", PGD, paramPreset_PGD2),
    ("DI", PGD, paramPreset_PGD3),
    ("DI", PGD, paramPreset_PGD4),
]

# Results on ArcFace
suffix = "ArcFace, R100, MS1M-V3, PGD"
torch.backends.cudnn.benchmark = True
dat = test_pgd(arcface, [("LFW", lfw)], "E", device, attack_configs = attack_configs, suffix = suffix)

# Results on Proposed
suffix = "Proposed, SLLNet, MS1M-V3, PGD"
torch.backends.cudnn.benchmark = True
dat = test_pgd(proposed, [("LFW", lfw)], "E", device, attack_configs = attack_configs, suffix = suffix)

#### Step 3-3. Test C&W Attack

In [ ]:
from attacks import CW
from eval_tools import test_cw

attack_configs = [
    ("U", CW, paramPreset_CW1),
    ("DI", CW, paramPreset_CW1),
]

# Results on ArcFace
suffix = "ArcFace, R100, MS1M-V3, CW"
torch.backends.cudnn.benchmark = True
dat = test_cw(arcface, [("LFW", lfw)], device, attack_configs = attack_configs, suffix = suffix, batch_size = 64)

# Results on Proposed
suffix = "Proposed, SLLNet, MS1M-V3, CW"
torch.backends.cudnn.benchmark = True
dat = test_cw(proposed, [("LFW", lfw)], device, attack_configs = attack_configs, suffix = suffix, batch_size = 64)